In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import tensorflow as tf
from keras.models import Sequential, Model
from tensorflow.keras import datasets
from tensorflow.keras.layers import Dense
from keras.layers import Input, Dense

In [ ]:
# 載入原本的模組
load_model = tf.keras.models.load_model('.\\mnist_basic_model.h5')
load_model.summary()

In [ ]:
# 這次用既有的訓練圖片來做
(training_images, training_labels), (test_images, test_labels) = datasets.mnist.load_data()
print("training_images.shape:",training_images.shape)
print("test_images.shape:",test_images.shape)

In [ ]:
# 把後面二維的部分攤平成一維
t_training_images = training_images.reshape(60000, 784)
t_test_images = test_images.reshape(10000, 784)

#轉換格式為 float32
t_training_images = t_training_images.astype('float32')
t_test_images = t_test_images.astype('float32')

# 將數值做正規化
t_training_images  = t_training_images / 255.0
t_test_images = t_test_images / 255.0

# 如果使用 sparse_categorical_crossentropy 就不需要做 One Hot Encoding 
# https://axk51013.medium.com/%E4%B8%8D%E8%A6%81%E5%86%8D%E5%81%9Aone-hot-encoding-b5126d3f8a63
num_classes=10
training_labels = tf.keras.utils.to_categorical(training_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

In [ ]:
# 挑選一個圖形當 base , 另一個圖形當作 target
base_index = 7
target_index = 4

fig, ax = plt.subplots(1,2)
ax[0].set_title(np.argmax(load_model(t_training_images[base_index].reshape(1,784))))
ax[0].imshow(training_images[base_index],cmap='gray')

ax[1].set_title(np.argmax(load_model(t_training_images[target_index].reshape(1,784))))
ax[1].imshow(training_images[target_index],cmap='gray')
plt.show()

In [ ]:
# 想辦法透過 Clean Label 演算法針對載入的圖檔做出生成式對抗樣本, 看能不能把這張圖變成外表看似 3, 其實內在是 9 

# 拿掉最後一層的 layer
# 參考資料 : https://www.tensorflow.org/guide/keras/sequential_model
new_model = Sequential()
new_model.add(Dense(128, input_dim=784, activation=tf.nn.relu))
new_model.add(Dense(10, activation='softmax'))
new_model.load_weights('mnist_basic_model.h5')

new_model.pop()
new_model.summary()

In [ ]:
# 先初始化一些參數
c=0.02

target_label = training_labels[target_index]
print(target_label)

t_base_image = t_training_images[base_index].reshape(1,784)
t_target_image = t_training_images[target_index].reshape(1,784)

# 這邊要看清楚, x 的初始值是給定 base image
x = tf.Variable(t_base_image)


beta = 6
learning_rate=0.001

In [ ]:
for i in range(1000) :
    
    with tf.GradientTape(persistent=True) as tape:  
        #loss1 = tf.keras.losses.MeanSquaredError(reduction='sum')(x,t_base_image)
        loss2 = tf.keras.losses.MeanSquaredError(reduction='sum')(new_model(x),new_model(t_target_image))
        #loss = loss1 + c*loss2
        loss = loss2

    gradients = tape.gradient(loss, x)    
    #print(gradients)
    
    tf.optimizers.Adam(learning_rate=learning_rate).apply_gradients(zip([gradients],[x]))
    x.assign((x+beta*learning_rate*t_base_image)/(1+beta*learning_rate))
    
    # 這邊依然要注意負值的問題
    x.assign(tf.clip_by_value(x,0,1))
    
    if i%50 == 0 :
        print(i)
        #print(gradients)
        #print('loss1:',loss1,' loss2:',loss2)
        print('loss2:',loss2)
        print(np.argmax(load_model(x.numpy())))


In [ ]:
hack_img = (x*255)
hack_img = tf.reshape(hack_img,[28,28])
hack_img = hack_img.numpy()

# 這邊要特別限制出來的數值, 應該可能會有負數
hack_img = np.clip(hack_img,0,255)
hack_img = hack_img.astype(np.uint8)
im = Image.fromarray(hack_img)
im.save("hack.bmp")

test_data = np.asarray(hack_img)
plt.imshow(test_data,cmap='gray')